In [58]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

In [45]:
# data = pd.read_csv('train_weather.csv')
names= ['yyyy','month','tmax','tmin']
df = pd.read_csv('train_weather.csv')

In [46]:
df

,yyyy month tmax tmin
0,1908 January 5.0 -1.4
1,1908 February 7.3 1.9
2,1908 March 6.2 0.3
3,1908 April Missing_1 2.1
4,1908 May Missing_2 7.7
...,...
438,1945 April 13.3 4.7
439,1945 May 15.4 6.9
440,1945 June 17.7 9.8
441,1945 July 20.0 12.1


In [47]:
df.columns

Index(['yyyy    month   tmax    tmin'], dtype='object')

In [48]:
df[['yyyy','month', 'tmax', 'tmin']] = df['yyyy    month   tmax    tmin'].str.split(expand=True) 

In [49]:
df = df.drop('yyyy    month   tmax    tmin', axis=1)

In [50]:
df2 = df[df['tmax'].str.contains('Missing')]
miss_max = df2.loc[:,df2.columns!='tmax']
df2 = df2.append(df[df['tmin'].str.contains('Missing')], ignore_index=True)
miss_min = df[df['tmin'].str.contains('Missing')]
miss_min = miss_min.loc[:,miss_min.columns!='tmin']

In [53]:
df2

,yyyy,month,tmax,tmin
0,1908,April,Missing_1,2.1
1,1908,May,Missing_2,7.7
2,1908,July,Missing_3,11.0
3,1910,February,Missing_5,1.5
4,1912,January,Missing_10,0.1
...,...,...,...,...
95,1942,February,2.1,Missing_94
96,1944,November,8.4,Missing_97
97,1945,January,3.7,Missing_98
98,1945,March,11.5,Missing_99


In [54]:
# drop missing max and min rows
df1 = pd.merge(df,df2, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
df1 = df1.reset_index(drop=True)

# convert text of months to digits to avoid error of string to float conversion
d = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6,
     'July':7, 'August':8, 'September':9, 'October':10, 'November':11,'December':12}

In [55]:
df1

,yyyy,month,tmax,tmin
0,1908,January,5.0,-1.4
1,1908,February,7.3,1.9
2,1908,March,6.2,0.3
3,1908,June,17.7,8.7
4,1908,August,17.5,9.7
...,...,...,...,...
338,1945,February,9.6,3.4
339,1945,April,13.3,4.7
340,1945,May,15.4,6.9
341,1945,June,17.7,9.8


In [56]:
df1.month = df1.month.map(d)
miss_max.month = miss_max.month.map(d)
miss_min.month = miss_min.month.map(d)
        
train = df1
test1 = miss_max
test2 = miss_min

In [38]:
# df[df['tmin'].str.contains('Missing')]

In [51]:
miss_min.head()

,yyyy,month,tmax
11,1908,December,5.8
26,1910,March,9.1
28,1910,May,14.3
36,1911,January,5.8
43,1911,August,21.6


In [52]:
miss_max.head()

,yyyy,month,tmin
3,1908,April,2.1
4,1908,May,7.7
6,1908,July,11.0
25,1910,February,1.5
48,1912,January,0.1


In [60]:
# to store predictions
r1  = []
clf1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.5, max_depth=2)
clf1.fit(train.loc[:,train.columns != 'tmax'],train.loc[:,'tmax'])

for i,j in enumerate(clf1.predict(test1)):
    r1.append((test1.index[i],round(j,1)))
    
clf2 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.5, max_depth=2)
clf2.fit(train.loc[:,train.columns != 'tmin'],train.loc[:,'tmin'])

for i,j in enumerate(clf2.predict(test2)):
    r1.append((test2.index[i],round(j,1)))
    
r1.sort()
for k in range(len(r1)):
    print(r1[k][1])

9.7
15.9
18.9
-0.6
6.5
2.5
5.8
0.1
12.1
5.1
0.3
9.8
14.8
18.0
2.6
6.2
11.1
7.1
8.9
13.9
3.6
18.4
11.7
6.2
-1.5
19.7
16.6
12.5
-0.8
18.3
4.1
10.2
8.4
19.1
18.4
15.3
5.2
-0.4
2.8
9.9
16.0
2.5
3.0
2.9
4.0
18.4
0.4
7.2
17.2
10.7
16.2
12.0
8.9
4.7
0.7
3.4
16.1
13.6
12.3
10.1
15.0
4.3
6.4
8.1
6.9
10.6
11.0
3.3
9.9
16.8
5.8
3.3
6.1
10.7
6.0
8.7
18.0
4.3
0.2
8.8
11.9
3.1
9.6
18.8
2.9
10.4
6.8
2.7
-0.9
6.9
4.6
11.1
13.5
-2.2
6.3
6.7
3.4
-1.3
3.8
11.7
